## CAFIR

CAFIR significa Cadastro de Imóveis Rurais. Através do site de [dados abertos do Governo Federal](https://dados.gov.br/home) é possível obter vários dados relacionados a tais imóveis, com exceção de informações protegidas pela Lei Geral de Proteção de Dados.

### Scraping

Para obtenção dos [dados](https://dadosabertos.rfb.gov.br/CAFIR/) será utilizado o arquivo na pasta Scripts chamado [scrap_csv.py](Scripts/scrap_csv.py), o qual irá baixar os CSVs referentes a dados Estaduais e os unirá em um único CSV.

### Limpeza de dados

O arquivo CSV resultante tem uma divisão fixa para o tamanho de cada coluna, muitos caracteres em brancos, datas com formato não padronizado e caracteres não-alfanuméricos, então é importante fazer uma "higiene" para facilitar sua inserção em bancos de dados e a visualização de seus dados.
O layout do CSV pode ser consultado também no site de [dados abertos](https://dadosabertos.rfb.gov.br/CAFIR/Layout%20Campos%20Dados%20Abertos%20Cafir.pdf), o qual utilizei como parâmetro para trocar o tamanho fixo de coluna por separadores. 
Para a limpeza dos dados foi utilizado o script [clean_csv.py](Scripts/clean_csv.py).
O resultado foi uma diminuição de quase 50% no tamanho do arquivo, que ajuda a diminuir o tempo para as queries futuras.

### SQL

Para analisar um volume tão grandes de dados é mais eficiente a utilização de um sistema de bancos de dados SQL, nesse caso em específico o Microsoft SQL. A tabela foi criada com base no layout fornecido [aqui](https://dadosabertos.rfb.gov.br/CAFIR/Layout%20Campos%20Dados%20Abertos%20Cafir.pdf). 

### Python
Serão necessárias as seguintes bibliotecas nesse projeto: Numpy, Pandas, Matplotlib, seaborn e pypyodbc. Abaixo está o comando para instalá-los.


In [1]:
pip install numpy pandas matplotlib seaborn pypyodbc

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\ciro\appdata\local\programs\python\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


Para começar, será testada a conexão com o servidor SQL na tabela CSVs_unidos que contém os dados do CAFIR

In [2]:
import pypyodbc as odbc

DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = r'DESKTOP-5V1VIGP\SQLEXPRESS'
DATABASE = 'Projeto CAFIR'
USERNAME = 'dbo'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE};
    UID={USERNAME};
    Trusted_Connection=yes;
"""

conn = odbc.connect(connection_string)
cursor = conn.cursor()

# Verificação da conexão
cursor.execute("SELECT COUNT(*) FROM CSVs_unidos;")
row = cursor.fetchone()
print(row[0])

conn.close()

9987664


## Análise sobre os dados 

### 1 - 
Buscando uma informação básica, realizei a soma de todos hectares dos imóveis ruras disponíveis no banco de dados.

In [4]:
import pypyodbc as odbc

DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = r'DESKTOP-5V1VIGP\SQLEXPRESS'
DATABASE = 'Projeto CAFIR'
USERNAME = 'dbo'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE};
    UID={USERNAME};
    Trusted_Connection=yes;
"""

conn = odbc.connect(connection_string)
cursor = conn.cursor()

cursor.execute(
    "SELECT SUM([Área total do imóvel (em hectares)])"
    "FROM [Projeto CAFIR].[dbo].[CSVs_unidos]"
)
sum = int(cursor.fetchone()[0])

print(f'A soma da área total dos imóveis é {sum} hectares')
conn.close()

A soma da área total dos imóveis é 25534799544 hectares


Porém o valor da soma, 25.534.799.544 hectares, é maior do que o território brasileiro, que possui cerca de 833.721.800 Hectares segundo o [IBGE](https://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/14318-asi-ibge-apresenta-nova-area-territorial-brasileira-8515767049-km). Portanto é seguro assumir que há registros que contam o mesmo território. Para continuar a investigação, será investigado quantas entradas estão duplicadas. O NIRF por definição na minha DB é um número único por entrada, mas para conferir, foi realizada a segunda pesquisa:

In [5]:
import pypyodbc as odbc

DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = r'DESKTOP-5V1VIGP\SQLEXPRESS'
DATABASE = 'Projeto CAFIR'
USERNAME = 'dbo'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE};
    UID={USERNAME};
    Trusted_Connection=yes;
"""

conn = odbc.connect(connection_string)
cursor = conn.cursor()

cursor.execute(
    "SELECT [NIRF - Número do imóvel na Receita Federal]"
    ", count(*)"
    "FROM [Projeto CAFIR].[dbo].[CSVs_unidos]"
    "GROUP BY [NIRF - Número do imóvel na Receita Federal]"
    "HAVING COUNT(*) > 1"
)
distinct = cursor.fetchone()

print(distinct)
conn.close()

None


Como esperado, não há duplicatas. Então é necessário por outro parâmetro de identificação, que é o código do imóvel no INCRA. Para não ficar muito extenso, nesse caderno mostrarei só as primeiras 20 linhas de resultados.

In [7]:
import pypyodbc as odbc

DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = r'DESKTOP-5V1VIGP\SQLEXPRESS'
DATABASE = 'Projeto CAFIR'
USERNAME = 'dbo'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE};
    UID={USERNAME};
    Trusted_Connection=yes;
"""

conn = odbc.connect(connection_string)
cursor = conn.cursor()

cursor.execute(
    "SELECT TOP 20 [Código do imóvel no Incra], COUNT(*) "
    "FROM [Projeto CAFIR].[dbo].[CSVs_unidos] "
    "GROUP BY [Código do imóvel no Incra] "
    "HAVING COUNT(*) > 1 "
    "ORDER BY [Código do imóvel no Incra]"
)

rows = cursor.fetchall()

for row in rows:
    print(f'Número do INCRA:{row[0]} - Quantidade de duplicatas: {row[1]}')

conn.close()

Número do INCRA: - Quantidade de duplicatas: 6337711
Número do INCRA:0000193771980 - Quantidade de duplicatas: 2
Número do INCRA:0000271103450 - Quantidade de duplicatas: 2
Número do INCRA:0000353450166 - Quantidade de duplicatas: 2
Número do INCRA:0000435650082 - Quantidade de duplicatas: 2
Número do INCRA:0000600479709 - Quantidade de duplicatas: 2
Número do INCRA:0000600951092 - Quantidade de duplicatas: 2
Número do INCRA:0000601952513 - Quantidade de duplicatas: 3
Número do INCRA:0010400372220 - Quantidade de duplicatas: 2
Número do INCRA:0010400524691 - Quantidade de duplicatas: 2
Número do INCRA:0011041287910 - Quantidade de duplicatas: 2
Número do INCRA:0011121522502 - Quantidade de duplicatas: 2
Número do INCRA:0011710046346 - Quantidade de duplicatas: 2
Número do INCRA:0240230178179 - Quantidade de duplicatas: 3
Número do INCRA:0450980214821 - Quantidade de duplicatas: 3
Número do INCRA:0480380537836 - Quantidade de duplicatas: 2
Número do INCRA:0480620001165 - Quantidade de d

Como é possível ver, há vários (6.337.711) imóveis sem cadastro no INCRA e pelo menos algumas duplicatas. Isso não deveria ser possível, mas é possível que os dados:
- Foram cadastrados erroneamente
- Não houve padronização na integração entre o sistema do CAFIR e do INCRA
- Há cadastro de propriedades sobrepostas, pois em regiões onde não há regularização fundiária plena, diferentes proprietários podem registrar o mesmo imóvel ou partes sobrepostas.
- O imóvel não possui cadastro no INCRA, o que implica em:
    - O imóvel pode estar registrado apenas para fins fiscais (imposto sobre a propriedade rural - ITR), mas nunca foi cadastrado no Sistema Nacional de Cadastro Rural (SNCR) do INCRA. O imóvel não terá um CCIR (Certificado de Cadastro de Imóvel Rural), que é obrigatório para várias operações legais, como compra, venda, hipoteca e regularização fundiária e também terá dificuldade em acessar programas de crédito rural ou outros benefícios que exigem o cadastro no INCRA.
    - Propriedade em áreas urbanas ou transição rural-urbana: O imóvel pode estar situado em uma área que foi convertida para zona urbana, mas permanece cadastrado no CAFIR para fins tributários. Nesse caso, o imóvel pode não ser mais exigido no SNCR (INCRA).
    - Pequenos imóveis rurais sem obrigatoriedade de cadastro no INCRA: Imóveis com áreas muito pequenas ou propriedades familiares podem não ter sido cadastrados no INCRA, especialmente se o proprietário não solicitou o CCIR. Isso ocorre com frequência em áreas onde o controle fundiário é menos rigoroso. Embora não seja exigido para pequenos imóveis em algumas circunstâncias, a ausência do cadastro no INCRA pode dificultar a regularização ou a transferência do imóvel no futuro.
    - Cadastro desatualizado ou em processo: O imóvel pode estar em processo de cadastro ou atualização no INCRA, mas já foi registrado no CAFIR para fins de tributação. Isso pode ocorrer quando o proprietário prioriza o cumprimento de obrigações fiscais antes de regularizar o cadastro fundiário.
